In [1]:
!pip install pandas tqdm

In [2]:
import pandas as pd
import shutil
import os
from tqdm import tqdm
import traceback

In [3]:
!sudo apt-get install lz4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lz4 is already the newest version (1.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [4]:
!gcloud auth login 


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  ^C


Command killed by keyboard interrupt



In [5]:
!gcloud config set project idc-external-025

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [6]:
data= pd.read_table('twoVM_2023_07_07_22_16.tsv')
data

entity:twoVM_2023_07_07_22_16_id  dcm2niixErrors  \
0                                     1             NaN   
1                                    10             NaN   
2                                   100             NaN   
3                                  1000             NaN   
4                                 10000             NaN   
...                                 ...             ...   
10503                              9995             NaN   
10504                              9996             NaN   
10505                              9997             NaN   
10506                              9998             NaN   
10507                              9999             NaN   

                  dicomsegAndRadiomicsSR_CompressedFiles  \
0      gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...   
1                                                    NaN   
2                                                    NaN   
3                                                    NaN   
4                                                    NaN   
...                                                  ...   
10503                                                NaN   
10504                                                NaN   
10505                                                NaN   
10506                                                NaN   
10507                                                NaN   

       dicomsegAndRadiomicsSR_Errors  \
0                                NaN   
1                                NaN   
2                                NaN   
3                                NaN   
4                                NaN   
...                              ...   
10503                            NaN   
10504                            NaN   
10505                            NaN   
10506                            NaN   
10507                            NaN   

                   dicomsegAndRadiomicsSR_OutputNotebook  \
0      gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...   
1                                                    NaN   
2                                                    NaN   
3                                                    NaN   
4                                                    NaN   
...                                                  ...   
10503                                                NaN   
10504                                                NaN   
10505                                                NaN   
10506                                                NaN   
10507                                                NaN   

                     dicomsegAndRadiomicsSR_UsageMetrics  dicomSegAndSRcpu  \
0      gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...                 4   
1                                                    NaN                 4   
2                                                    NaN                 4   
3                                                    NaN                 4   
4                                                    NaN                 8   
...                                                  ...               ...   
10503                                                NaN                 8   
10504                                                NaN                 4   
10505                                                NaN                 8   
10506                                                NaN                 8   
10507                                                NaN                 8   

       dicomSegAndSRram dicomToNiftiConverterTool  \
0                    16                  dcm2niix   
1                    16                  dcm2niix   
2                    16                  dcm2niix   
3                    16                  dcm2niix   
4                    32                  dcm2niix   
...                 ...                       ...   
10503                32                  dcm2niix   
10504         

In [9]:

dicom_seg_download_urls = data.dropna(subset=['dicomsegAndRadiomicsSR_CompressedFiles'])['dicomsegAndRadiomicsSR_CompressedFiles'].to_list()
dicom_seg_download_urls

['gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/f30c1eb1-9627-4a22-84e5-3032dc7f42c1/TotalSegmentator/c1c9e45e-b4d4-4c3d-9de0-fa194719fdc0/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4']

In [ ]:
# !gsutil cp gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/09254959-5ee1-48ed-863c-895e385abef0/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4 .

In [10]:
for url in tqdm(dicom_seg_download_urls):
    print('processing url:'+url)   
    try:
      shutil.rmtree(f'dicom_seg_objects')  
      shutil.rmtree(f'itkimage2segimage')
    except OSError: 
      pass
    try:    
        os.mkdir(f'dicom_seg_objects')      
        !gsutil cp {url} . > /dev/null 2>&1
        !lz4 -d --rm dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4 -c | tar --strip-components=1  -xvf - > /dev/null 2>&1
        !find ./itkimage2segimage -name '*.dcm.lz4' -exec mv -t dicom_seg_objects {} + > /dev/null 2>&1
        !lz4 -d -m --rm "dicom_seg_objects"/*.lz4 > /dev/null 2>&1
        !gsutil -m cp -r dicom_seg_objects/* gs://totalsegmentator-test/DICOM_SEGS/ > /dev/null 2>&1
    except Exception as e:
        print(f'Error processing {url}: {e}')
        traceback.print_exc()    
shutil.rmtree(f'dicom_seg_objects')     
shutil.rmtree(f'itkimage2segimage')   

  0%|          | 0/1 [00:00<?, ?it/s]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/f30c1eb1-9627-4a22-84e5-3032dc7f42c1/TotalSegmentator/c1c9e45e-b4d4-4c3d-9de0-fa194719fdc0/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4


100%|██████████| 1/1 [00:11<00:00, 11.79s/it]


In [11]:
#sr_download_urls=data['structuredReportsDICOM'].to_list()
sr_download_urls = data.dropna(subset=['structuredReportsDICOM'])['structuredReportsDICOM'].to_list()
sr_download_urls
#sr_download_urls=['gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/7b746cb8-3569-4d07-8664-3388424d58da/TotalSegmentator/41ee65de-087c-470e-a22d-c79bb4aab1cc/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4']

['gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/f30c1eb1-9627-4a22-84e5-3032dc7f42c1/TotalSegmentator/c1c9e45e-b4d4-4c3d-9de0-fa194719fdc0/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4']

In [ ]:
# for url in tqdm(sr_download_urls):
#     print('processing url:' + url)
#     try:
#         shutil.rmtree(f'structuredReportsDICOM')
#         shutil.rmtree(f'decompressedStructuredReportsDICOM')
#     except OSError:
#         pass
#     os.mkdir(f'decompressedStructuredReportsDICOM')
#     try:
#         !gsutil cp {url} . > /dev/null 2>&1
#         !lz4 -d --rm structuredReportsDICOM.tar.lz4 -c | tar --strip-components=1  -xvf - > /dev/null 2>&1
#         !find ./structuredReportsDICOM -name '*.dcm.lz4' -exec mv -t decompressedStructuredReportsDICOM {} + > /dev/null 2>&1
#         !lz4 -d -m --rm "decompressedStructuredReportsDICOM"/*.lz4 > /dev/null 2>&1
#         !gsutil -m cp -r decompressedStructuredReportsDICOM/* gs://total_segmentator_nlst_sample_061023/decompressedStructuredReportsDICOM/ > /dev/null 2>&1
#     except Exception as e:
#         print(f'Error processing {url}: {e}')
#         traceback.print_exc()

In [12]:
import os
import shutil
import pandas as pd

# Create an empty DataFrame to store the series IDs
series_df = pd.DataFrame(columns=['series_id'])

batch_count = 1  # Counter for batch folders

for url in tqdm(sr_download_urls):
    print('processing url:' + url)
    try:
        shutil.rmtree(f'structuredReportsDICOM')
        shutil.rmtree(f'decompressedStructuredReportsDICOM')
    except OSError:
        pass
    os.mkdir(f'decompressedStructuredReportsDICOM')
    try:
        !gsutil cp {url} . > /dev/null 2>&1
        !lz4 -d --rm structuredReportsDICOM.tar.lz4 -c | tar --strip-components=1 -xvf - > /dev/null 2>&1
        !find ./structuredReportsDICOM -name '*.dcm.lz4' -exec mv -t decompressedStructuredReportsDICOM {} + > /dev/null 2>&1
        !lz4 -d -m --rm "decompressedStructuredReportsDICOM"/*.lz4 > /dev/null 2>&1
        !gsutil -m cp -r decompressedStructuredReportsDICOM/* gs://totalsegmentator-test/decompressedStructuredReportsDICOM/batch_{batch_count}/ > /dev/null 2>&1

        # Find all series IDs and add them to the DataFrame
        series_ids = [filename.split('_')[0] for filename in os.listdir('decompressedStructuredReportsDICOM')]
        url_series_df = pd.DataFrame({'series_id': series_ids})

        # Append the current DataFrame to the main DataFrame
        series_df = pd.concat([series_df, url_series_df], ignore_index=True)

    except Exception as e:
        print(f'Error processing {url}: {e}')
        traceback.print_exc()

    try:
        # Upload shape_sr.dcm files in the batch
        gcs_uri = f"gs://totalsegmentator-test/decompressedStructuredReportsDICOM/batch_{batch_count}/*.dcm"
        !gcloud healthcare dicom-stores import gcs test --dataset=totalsegmentator-test --location=us-central1 --gcs-uri={gcs_uri}
    except Exception as e:
        print(f'Error processing {url}: {e}')
        traceback.print_exc()

    # Increment the batch counter
    batch_count += 1


  0%|          | 0/1 [00:00<?, ?it/s]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/f30c1eb1-9627-4a22-84e5-3032dc7f42c1/TotalSegmentator/c1c9e45e-b4d4-4c3d-9de0-fa194719fdc0/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [test]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/totalsegmentator-test/operations/16186470812193128449] to complete...done.    
name: projects/idc-external-025/locations/us-central1/datasets/totalsegmentator-test/dicomStores/test


100%|██████████| 1/1 [00:22<00:00, 22.61s/it]


In [ ]:
shutil.rmtree(f'structuredReportsDICOM') 

In [ ]:
# sr_download_urls=data['structuredReportsDICOM'].to_list()
# try:
#   shutil.rmtree(f'structuredReportsDICOM')
#   shutil.rmtree(f'decompressedStructuredReportsDICOM')
# except OSError: 
#   pass 
# os.mkdir(f'decompressedStructuredReportsDICOM')

# for url in sr_download_urls:
#   print('processing '+url)   
#   !gsutil cp {url} .
#   !lz4 -d --rm structuredReportsDICOM.tar.lz4 -c | tar --strip-components=1  -xvf -
#   !find ./structuredReportsDICOM -name '*.dcm.lz4' -exec mv -t decompressedStructuredReportsDICOM {} +
#   #!find dicom_seg_objects -type f -name '*.lz4' -exec sh -c 'lz4 -d --rm "$1"' _ {} \;



In [ ]:
# import pandas as pd
# filenames = pd.read_csv('filenames.txt', header=None)[0].to_list()
# filenames

In [ ]:
for file in filenames:
 print(file)   
 !gcloud healthcare dicom-stores import gcs trial --dataset=total_segmentator_nlst_sample_060823 --location=us-central1 --gcs-uri=gs://total_segmentator_nlst_sample_060823/decompressedStructuredReportsDICOM/{file}_shape_sr.dcm
 !gcloud healthcare dicom-stores import gcs trial --dataset=total_segmentator_nlst_sample_060823 --location=us-central1 --gcs-uri=gs://total_segmentator_nlst_sample_060823/decompressedStructuredReportsDICOM/{file}_firstorder_sr.dcm
         